In [1]:
!pip install ultralytics
!pip install minio




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.4 MB/s eta 0:00:00


In [2]:
from minio import Minio


client = Minio(
    endpoint="minio.smtornado.ru",
    access_key=None,
    secret_key=None,
    secure=True
)

bucket_name = "datasets"
file_name = "dataset_yolo.zip"
destination = "./dataset_yolo.zip"

try:
    client.fget_object(bucket_name, file_name, destination)
    print(f"File '{file_name}' downloaded successfully to '{destination}'.")
except Exception as e:
    print(f"Error occurred: {e}")

File 'dataset_yolo.zip' downloaded successfully to './dataset_yolo.zip'.


In [3]:
import torch
cuda = torch.device('cuda')

In [6]:
from ultralytics import YOLO

model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model on the yolo_dataset  for 100 epochs
results = model.train(data="./dataset_yolo.zip", epochs=2, imgsz=[768, 384], plots=True, device=cuda, conf=0.2)



engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=./dataset_yolo.zip, epochs=2, time=None, patience=100, batch=16, imgsz=[768, 384], save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=Fals

train: Scanning /content/datasets/dataset_yolo/train/labels.cache... 6332 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6332/6332 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/datasets/dataset_yolo/valid/labels.cache... 930 images, 0 backgrounds, 0 corrupt: 100%|██████████| 930/930 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 768 train, 768 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2      3.49G      2.286       2.15      2.044         34        768: 100%|██████████| 396/396 [02:53<00:00,  2.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:05<00:00,  5.02it/s]

                   all        930       1068       0.35       0.33      0.343      0.128



2 epochs completed in 0.103 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 5.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:09<00:00,  3.30it/s]


                   all        930       1068      0.349      0.329      0.342      0.127
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train3


In [7]:
from PIL import Image
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy arrayii
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    r.show()

    # Save results to disk
    r.save(filename=f"results{i}.jpg")

TypeError: 'DetMetrics' object is not iterable

In [1]:
import astropy
from astropy.io import fits
hdul = fits.open('image.fits')

In [9]:
head = hdul[1].header

In [11]:
head['R_SUN']

1628.2066650390625

In [3]:
def decorator(func):
  def wrapper():
    print("Do something before func")
    func()
    print("Do something after func")
  return wrapper

@decorator
def hello_world():
  print("Hello world!")

hello_world()

Do something before func
Hello world!
Do something after func
